In [3]:
print("sahana")

sahana


In [ ]:
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

In [ ]:
sa = np.array([[1, 2, 3], [4, 5, 6,]])
ha = np.array([[1, 2, 3], [4, 5, 6,]])
saha = np.dstack([sa, ha])
print(saha.shape)




In [ ]:
dir_name = "D:\\processed"
X = []
y = []
Y = []
files = os.listdir(dir_name)
count = 0
for file in files:
    if file.endswith(".csv"):
        count = count+1
        print(file)
        filepath = dir_name + "\\" + file
        open_face_data = pd.read_csv(filepath)
        au_scores = open_face_data.loc[:, ["frame", " confidence", " AU01_r", " AU02_r", " AU04_r", " AU05_r", " AU06_r",
                                           " AU07_r", " AU09_r", " AU10_r", " AU12_r", " AU14_r",
                                           " AU15_r", " AU17_r", " AU20_r", " AU23_r", " AU25_r", " AU26_r"]]
        confident = au_scores[au_scores[" confidence"] > 0.8]
        if confident.shape[0] > 35:
            samples = confident.sample(n=35).sort_values(by="frame").drop(columns = ['frame', ' confidence'])
            weights = file.split('_')
            weight = weights[1].rstrip("lbs")
            weight = float(weight)

            # find the weights and accordingly decide the label
            if weight == 7.5:
                if count == 1:
                    y = np.full(35, 0).T
                else :
                    y = np.dstack([y, np.full(35, 0).T])
                Y.append(0)
                # y_train.append('light')
            elif weight == 25:
                if count == 1:
                    y = np.full(35, 1).T
                else :
                    y = np.dstack([y, np.array(np.full(35, 1).T)])
                Y.append(1)
                # y_train.append('light')
            else:
                if count == 1:
                    y = np.full(35, 2).T
                else :
                    y = np.dstack([y, np.full(35, 2).T])
                Y.append(2)
                # y_train.append('light')
                
            if count == 1:
                X = samples.to_numpy()
                
            else:
                print(samples.to_numpy().shape)
                print(X.shape)
                X = np.dstack([X, samples.to_numpy()])

In [12]:
real_X = X.reshape((X.shape[2], 35, 16))
print(y.shape)
real_y = y.reshape((y.shape[2], 35, 1))
Y = np.array(Y)


(3594, 35, 18)


In [ ]:
model = Sequential()
model.add(LSTM(80, activation='relu', input_shape=(35, 16)))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
print(model.summary())

In [ ]:
model.fit(real_X, real_y, epochs=2000, validation_split=0.2, batch_size=5)

In [ ]:
model_1 = Sequential()
model_1.add(LSTM(50, activation='relu', input_shape=(35, 16)))
model_1.add(Dense(1))
model_1.compile(optimizer='adam', loss='mse')
history = model_1.fit(real_X, Y, epochs=1000, validation_split=0.2, verbose=1)

